In [1]:
import spacy
import numpy as np
import re
nlp = spacy.load('en_core_web_sm')

In [26]:
def nGrams(doc_string_or_list,n,docIsAlreadyTokenised):
    doc = doc_string_or_list
    if not docIsAlreadyTokenised:
        doc = [token.text for token in nlp(doc_string_or_list)]
    doc = ' '.join(doc).lower().split(' ')
    grams = [doc[i:i+n] for i in range(len(doc)-n+1)]
    return grams

In [12]:
doc = 'how old are you today or can you tell me something about today'
n = 3
grams = nGrams(doc,n,False)

In [13]:
grams

[['how', 'old', 'are'],
 ['old', 'are', 'you'],
 ['are', 'you', 'today'],
 ['you', 'today', 'or'],
 ['today', 'or', 'can'],
 ['or', 'can', 'you'],
 ['can', 'you', 'tell'],
 ['you', 'tell', 'me'],
 ['tell', 'me', 'something'],
 ['me', 'something', 'about'],
 ['something', 'about', 'today']]

In [14]:
from collections import defaultdict

In [15]:
def buildModel():
    model = defaultdict(lambda: defaultdict(lambda: 0))
    return model

In [16]:
buildModel()

defaultdict(<function __main__.buildModel.<locals>.<lambda>()>, {})

In [17]:
def updateCount(nGram,model):
    w_1_to_n_minus_1 = tuple(nGram[:-1])
    w_n = nGram[-1]
    model[w_1_to_n_minus_1][w_n]+=1
    return model

In [38]:
def computeProbability(model):
    for w_1_to_n_minus_1 in model:
        totalCount = float(sum(model[w_1_to_n_minus_1].values()))
        for w_n in model[w_1_to_n_minus_1]:
            model[w_1_to_n_minus_1][w_n] /= totalCount
    return model

In [19]:
pip install dill

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [20]:
import dill

In [21]:
def saveModel(model,fileNameDotpkl):
    with open(fileNameDotpkl,'wb') as f:
        dill.dump(model,f)
def loadModel(fileNameDotpkl):
    with open(fileNameDotpkl,'r') as f:
        model = dill.load(f)
    return model

In [33]:
from sklearn.datasets import fetch_20newsgroups as getData
from nltk.corpus import reuters as corpus

In [24]:
X,y = getData(subset='train',remove=('headers','footers','coutes'),return_X_y=True)

In [25]:
n = 3
model = buildModel()


In [28]:
for doc in X:
    for nGram in nGrams(doc,n,False):
            model = updateCount(nGram,model)

In [29]:
saveModel(model,'dict_model.pkl')

In [35]:
for sentence in corpus.sents():
       for nGram in nGrams(sentence,n,True):
            model = updateCount(nGram,model)

In [39]:
model = computeProbability(model)
saveModel(model,'model.pkl')

In [50]:
text = ['after','that']
nextWords = list(model[tuple(text[-n+1:])].keys())
probs = list(model[tuple(text[-n+1:])].values())
nextword = np.random.choice(nextWords,1,probs)[0]
nextword

';'

In [45]:
nextWords

dict_keys(['.', ',', 'date', 'the', 'i', 'he', 'it', 'time', '\n\t', 'and', 'we', 'they', 'oh', 'post', 'thou', 'tribulation', 'step', 'or', 'space', 'night', '$', 'those', 'only', ';', '\n', 'of', '!', 'at', 'meeting', 'will'])

In [46]:
probs

[0.12,
 0.30666666666666664,
 0.02666666666666667,
 0.02666666666666667,
 0.08,
 0.05333333333333334,
 0.013333333333333334,
 0.013333333333333334,
 0.013333333333333334,
 0.02666666666666667,
 0.02666666666666667,
 0.013333333333333334,
 0.04,
 0.013333333333333334,
 0.013333333333333334,
 0.013333333333333334,
 0.013333333333333334,
 0.013333333333333334,
 0.013333333333333334,
 0.013333333333333334,
 0.013333333333333334,
 0.013333333333333334,
 0.013333333333333334,
 0.013333333333333334,
 0.013333333333333334,
 0.013333333333333334,
 0.013333333333333334,
 0.013333333333333334,
 0.02666666666666667,
 0.013333333333333334]

In [131]:
def sampleText(model,starting_text=['after','that'],max_length=100,nGramsize=3):
    text = starting_text
    n = nGramsize
    while not len(text)>max_length:
        nextWords = list(model[tuple(text[-n+1:])].keys())
        probs = list(model[tuple(text[-n+1:])].values())
        if len(nextWords)>0:
            nextword = np.argmax(probs)
#            print(nextword)
            text.append(nextWords[nextword])
        else:
            break
    sampled = ' '.join(text)
    return sampled

In [132]:
sampleText(model)


"after that , i 'm not sure if this is a \n > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > >"